In [0]:
import pandas as pd
import glob
import math

import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from __future__ import division
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn import tree, linear_model
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import explained_variance_score, mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
#SVR sklearn
#Per el dia 3 enviar un report del que he anat fent.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [0]:
PATH_TO_DATA = '/content/drive/My Drive/MATT/MLEARN/MLEARN.zip (Unzipped Files)/MLEARN/00Data'

In [0]:
data = pd.read_csv(PATH_TO_DATA+'/Generated/DB20200101.csv')

In [0]:
print(data.dtypes.unique())

[dtype('int64') dtype('float64') dtype('O')]


In [0]:
data.head()

,Unnamed: 0,B365H,B365D,B365A,FTR,Date,HomeTeam,AwayTeam,HomeTeamPercPoints,HomeTeamGolsFavorMean,HomeTeamGolsFavorMedian,HomeTeamGolsFavorMax,HomeTeamGolsContraMean,HomeTeamGolsContraMedian,HomeTeamGolsContraMax,HomeTeam_Position,HomeTeam_Home_Wins_season,HomeTeam_Away_Wins_season,HomeTeam_Wins_season,HomeTeam_Home_Lose_season,HomeTeam_Away_Lose_season,HomeTeam_Lose_season,HomeTeam_Home_Draw_season,HomeTeam_Away_Draw_season,HomeTeam_Draw_season,HomeTeam_Points,HomeTeam_Home_goals_avg,HomeTeam_Home_goals_med,HomeTeam_Home_goals_max,HomeTeam_Home_goals_mode,HomeTeam_Away_goals_avg,HomeTeam_Away_goals_med,HomeTeam_Away_goals_max,HomeTeam_Away_goals_mode,HomeTeam_AgeMean,HomeTeam_AgeMedian,HomeTeam_DefensaMax,HomeTeam_DefensaMean,HomeTeam_DefensaMedian,HomeTeam_DelanteroMax,HomeTeam_DelanteroMean,HomeTeam_DelanteroMedian,HomeTeam_MedioMax,HomeTeam_MedioMean,HomeTeam_MedioMedian,HomeTeam_OverallMax,HomeTeam_OverallMean,HomeTeam_OverallMedian,HomeTeam_PorteroMax,HomeTeam_PorteroMean,HomeTeam_PorteroMedian,HomeTeam_TopOverallMean,HomeTeam_TopOverallMedian,HomeTeam_TopOverallMin,AwayTeamPercPoints,AwayTeamGolsFavorMean,AwayTeamGolsFavorMedian,AwayTeamGolsFavorMax,AwayTeamGolsContraMean,AwayTeamGolsContraMedian,AwayTeamGolsContraMax,AwayTeam_Position,AwayTeam_Home_Wins_season,AwayTeam_Away_Wins_season,AwayTeam_Wins_season,AwayTeam_Home_Lose_season,AwayTeam_Away_Lose_season,AwayTeam_Lose_season,AwayTeam_Home_Draw_season,AwayTeam_Away_Draw_season,AwayTeam_Draw_season,AwayTeam_Points,AwayTeam_Home_goals_avg,AwayTeam_Home_goals_med,AwayTeam_Home_goals_max,AwayTeam_Home_goals_mode,AwayTeam_Away_goals_avg,AwayTeam_Away_goals_med,AwayTeam_Away_goals_max,AwayTeam_Away_goals_mode,AwayTeam_AgeMean,AwayTeam_AgeMedian,AwayTeam_DefensaMax,AwayTeam_DefensaMean,AwayTeam_DefensaMedian,AwayTeam_DelanteroMax,AwayTeam_DelanteroMean,AwayTeam_DelanteroMedian,AwayTeam_MedioMax,AwayTeam_MedioMean,AwayTeam_MedioMedian,AwayTeam_OverallMax,AwayTeam_OverallMean,AwayTeam_OverallMedian,AwayTeam_PorteroMax,AwayTeam_PorteroMean,AwayTeam_PorteroMedian,AwayTeam_TopOverallMean,AwayTeam_TopOverallMedian,AwayTeam_TopOverallMin
0,0,2.2,3.2,3.50,D,26/08/2016,Betis,La Coruna,0.000000,2.0,2.0,2,6.0,6.0,6,10,6,5,11,7,8,15,6,6,12,45,0.894737,1.0,2,1,0.894737,1.0,3,1,26.142857,25.0,78.0,73.555556,73.0,81.0,71.400000,75.0,79.0,74.818182,76.0,81.0,73.678571,75.5,81.0,73.666667,70.0,78.181818,78.0,60.0,1.000000,2.0,2.0,2,1.0,1.0,1,15,4,4,8,7,5,12,8,10,18,42,1.315789,1.0,3,2,1.052632,1.0,3,1,25.666667,25.0,79.0,74.625000,75.0,79.0,68.500000,67.0,79.0,72.062500,74.0,79.0,72.151515,75.0,75.0,73.333333,75.0,77.272727,77.0,61.0
1,1,2.3,3.1,3.40,D,26/08/2016,Espanol,Malaga,0.000000,4.0,4.0,4,6.0,6.0,6,14,9,3,12,5,14,19,5,2,7,43,1.157895,1.0,4,1,0.947368,1.0,4,0,26.142857,26.0,77.0,73.666667,76.0,80.0,73.000000,75.0,79.0,75.300000,77.0,82.0,74.250000,76.5,82.0,75.000000,79.0,78.454545,78.0,64.0,0.333333,1.0,1.0,1,1.0,1.0,1,9,8,4,12,5,9,14,6,6,12,48,1.368421,1.0,4,1,0.631579,0.0,3,0,25.000000,24.0,78.0,73.181818,75.0,77.0,68.875000,71.0,81.0,75.909091,76.0,81.0,73.090909,75.0,81.0,73.666667,78.0,78.272727,78.0,58.0
2,2,2.7,3.2,2.75,H,27/08/2016,Eibar,Valencia,0.000000,1.0,1.0,1,2.0,2.0,2,13,8,3,11,6,11,17,5,5,10,43,1.368421,1.0,5,1,1.210526,1.0,4,0,25.096774,25.0,79.0,72.000000,75.0,78.0,70.500000,72.5,79.0,73.900000,77.0,79.0,71.870968,75.0,75.0,68.500000,69.5,77.545455,78.0,60.0,0.000000,2.0,2.0,2,4.0,4.0,4,12,6,5,11,6,10,16,7,4,11,44,1.315789,1.0,4,0,1.105263,1.0,5,0,24.333333,24.0,83.0,74.166667,76.5,83.0,76.375000,76.5,81.0,71.400000,72.5,83.0,73.878788,76.0,82.0,74.333333,75.0,80.181818,79.0,61.0
3,3,8.5,3.6,1.53,D,27/08/2016,Leganes,Ath Madrid,1.000000,1.0,1.0,1,0.0,0.0,0,19,6,3,9,6,14,20,7,2,9,36,1.210526,1.0,4,1,0.736842,0.0,2,0,25.454545,25.0,76.0,70.909091,74.0,73.0,69.500000,71.0,77.0,73.500000,74.0,77.0,71.939394,73.0,76.0,71.750000,73.0,76.181818,76.0,61.0,0.333333,1.0,1.0,1,1.0,1.0,1,3,15,13,28,1,5,6,3,1,4,88,1.736842,1.0,5,1,1.578947,1.0,3,1,25.419355,24.0,88.0,77.666667,

In [0]:
data = data.drop(columns=['Unnamed: 0','Date','HomeTeam','AwayTeam'])#,'B365H','B365D','B365A'

In [0]:
data['B365H'] = 1/data['B365H']
data['B365D'] = 1/data['B365D']
data['B365A'] = 1/data['B365A']
data['Sum'] = data['B365H'] + data['B365D'] + data['B365A'] 
data['B365H'] = data['B365H']/data['Sum']
data['B365D'] = data['B365D']/data['Sum']
data['B365A'] = data['B365A']/data['Sum']
del data['Sum']

In [0]:
data.shape

(1199, 96)

In [0]:
data.head()

,B365H,B365D,B365A,FTR,HomeTeamPercPoints,HomeTeamGolsFavorMean,HomeTeamGolsFavorMedian,HomeTeamGolsFavorMax,HomeTeamGolsContraMean,HomeTeamGolsContraMedian,HomeTeamGolsContraMax,HomeTeam_Position,HomeTeam_Home_Wins_season,HomeTeam_Away_Wins_season,HomeTeam_Wins_season,HomeTeam_Home_Lose_season,HomeTeam_Away_Lose_season,HomeTeam_Lose_season,HomeTeam_Home_Draw_season,HomeTeam_Away_Draw_season,HomeTeam_Draw_season,HomeTeam_Points,HomeTeam_Home_goals_avg,HomeTeam_Home_goals_med,HomeTeam_Home_goals_max,HomeTeam_Home_goals_mode,HomeTeam_Away_goals_avg,HomeTeam_Away_goals_med,HomeTeam_Away_goals_max,HomeTeam_Away_goals_mode,HomeTeam_AgeMean,HomeTeam_AgeMedian,HomeTeam_DefensaMax,HomeTeam_DefensaMean,HomeTeam_DefensaMedian,HomeTeam_DelanteroMax,HomeTeam_DelanteroMean,HomeTeam_DelanteroMedian,HomeTeam_MedioMax,HomeTeam_MedioMean,HomeTeam_MedioMedian,HomeTeam_OverallMax,HomeTeam_OverallMean,HomeTeam_OverallMedian,HomeTeam_PorteroMax,HomeTeam_PorteroMean,HomeTeam_PorteroMedian,HomeTeam_TopOverallMean,HomeTeam_TopOverallMedian,HomeTeam_TopOverallMin,AwayTeamPercPoints,AwayTeamGolsFavorMean,AwayTeamGolsFavorMedian,AwayTeamGolsFavorMax,AwayTeamGolsContraMean,AwayTeamGolsContraMedian,AwayTeamGolsContraMax,AwayTeam_Position,AwayTeam_Home_Wins_season,AwayTeam_Away_Wins_season,AwayTeam_Wins_season,AwayTeam_Home_Lose_season,AwayTeam_Away_Lose_season,AwayTeam_Lose_season,AwayTeam_Home_Draw_season,AwayTeam_Away_Draw_season,AwayTeam_Draw_season,AwayTeam_Points,AwayTeam_Home_goals_avg,AwayTeam_Home_goals_med,AwayTeam_Home_goals_max,AwayTeam_Home_goals_mode,AwayTeam_Away_goals_avg,AwayTeam_Away_goals_med,AwayTeam_Away_goals_max,AwayTeam_Away_goals_mode,AwayTeam_AgeMean,AwayTeam_AgeMedian,AwayTeam_DefensaMax,AwayTeam_DefensaMean,AwayTeam_DefensaMedian,AwayTeam_DelanteroMax,AwayTeam_DelanteroMean,AwayTeam_DelanteroMedian,AwayTeam_MedioMax,AwayTeam_MedioMean,AwayTeam_MedioMedian,AwayTeam_OverallMax,AwayTeam_OverallMean,AwayTeam_OverallMedian,AwayTeam_PorteroMax,AwayTeam_PorteroMean,AwayTeam_PorteroMedian,AwayTeam_TopOverallMean,AwayTeam_TopOverallMedian,AwayTeam_TopOverallMin
0,0.431766,0.296839,0.271396,D,0.000000,2.0,2.0,2,6.0,6.0,6,10,6,5,11,7,8,15,6,6,12,45,0.894737,1.0,2,1,0.894737,1.0,3,1,26.142857,25.0,78.0,73.555556,73.0,81.0,71.400000,75.0,79.0,74.818182,76.0,81.0,73.678571,75.5,81.0,73.666667,70.0,78.181818,78.0,60.0,1.000000,2.0,2.0,2,1.0,1.0,1,15,4,4,8,7,5,12,8,10,18,42,1.315789,1.0,3,2,1.052632,1.0,3,1,25.666667,25.0,79.0,74.625000,75.0,79.0,68.500000,67.0,79.0,72.062500,74.0,79.0,72.151515,75.0,75.0,73.333333,75.0,77.272727,77.0,61.0
1,0.413495,0.306787,0.279718,D,0.000000,4.0,4.0,4,6.0,6.0,6,14,9,3,12,5,14,19,5,2,7,43,1.157895,1.0,4,1,0.947368,1.0,4,0,26.142857,26.0,77.0,73.666667,76.0,80.0,73.000000,75.0,79.0,75.300000,77.0,82.0,74.250000,76.5,82.0,75.000000,79.0,78.454545,78.0,64.0,0.333333,1.0,1.0,1,1.0,1.0,1,9,8,4,12,5,9,14,6,6,12,48,1.368421,1.0,4,1,0.631579,0.0,3,0,25.000000,24.0,78.0,73.181818,75.0,77.0,68.875000,71.0,81.0,75.909091,76.0,81.0,73.090909,75.0,81.0,73.666667,78.0,78.272727,78.0,58.0
2,0.353911,0.298613,0.347476,H,0.000000,1.0,1.0,1,2.0,2.0,2,13,8,3,11,6,11,17,5,5,10,43,1.368421,1.0,5,1,1.210526,1.0,4,0,25.096774,25.0,79.0,72.000000,75.0,78.0,70.500000,72.5,79.0,73.900000,77.0,79.0,71.870968,75.0,75.0,68.500000,69.5,77.545455,78.0,60.0,0.000000,2.0,2.0,2,4.0,4.0,4,12,6,5,11,6,10,16,7,4,11,44,1.315789,1.0,4,0,1.105263,1.0,5,0,24.333333,24.0,83.0,74.166667,76.5,83.0,76.375000,76.5,81.0,71.400000,72.5,83.0,73.878788,76.0,82.0,74.333333,75.0,80.181818,79.0,61.0
3,0.112150,0.264798,0.623053,D,1.000000,1.0,1.0,1,0.0,0.0,0,19,6,3,9,6,14,20,7,2,9,36,1.210526,1.0,4,1,0.736842,0.0,2,0,25.454545,25.0,76.0,70.909091,74.0,73.0,69.500000,71.0,77.0,73.500000,74.0,77.0,71.939394,73.0,76.0,71.750000,73.0,76.181818,76.0,61.0,0.333333,1.0,1.0,1,1.0,1.0,1,3,15,13,28,1,5,6,3,1,4,88,1.736842,1.0,5,1,1.578947,1.0,3,1,25.419355,24.0,88.0,77.666667,82.0,88.0,75.666667,80.0,84.0,76.416667,80.5,88.0,76.548387,81.0,87.0,75.750000,75.5,84.636364,

In [0]:
X = data.iloc[:,4:]

In [0]:
X.head()

,HomeTeamPercPoints,HomeTeamGolsFavorMean,HomeTeamGolsFavorMedian,HomeTeamGolsFavorMax,HomeTeamGolsContraMean,HomeTeamGolsContraMedian,HomeTeamGolsContraMax,HomeTeam_Position,HomeTeam_Home_Wins_season,HomeTeam_Away_Wins_season,HomeTeam_Wins_season,HomeTeam_Home_Lose_season,HomeTeam_Away_Lose_season,HomeTeam_Lose_season,HomeTeam_Home_Draw_season,HomeTeam_Away_Draw_season,HomeTeam_Draw_season,HomeTeam_Points,HomeTeam_Home_goals_avg,HomeTeam_Home_goals_med,HomeTeam_Home_goals_max,HomeTeam_Home_goals_mode,HomeTeam_Away_goals_avg,HomeTeam_Away_goals_med,HomeTeam_Away_goals_max,HomeTeam_Away_goals_mode,HomeTeam_AgeMean,HomeTeam_AgeMedian,HomeTeam_DefensaMax,HomeTeam_DefensaMean,HomeTeam_DefensaMedian,HomeTeam_DelanteroMax,HomeTeam_DelanteroMean,HomeTeam_DelanteroMedian,HomeTeam_MedioMax,HomeTeam_MedioMean,HomeTeam_MedioMedian,HomeTeam_OverallMax,HomeTeam_OverallMean,HomeTeam_OverallMedian,HomeTeam_PorteroMax,HomeTeam_PorteroMean,HomeTeam_PorteroMedian,HomeTeam_TopOverallMean,HomeTeam_TopOverallMedian,HomeTeam_TopOverallMin,AwayTeamPercPoints,AwayTeamGolsFavorMean,AwayTeamGolsFavorMedian,AwayTeamGolsFavorMax,AwayTeamGolsContraMean,AwayTeamGolsContraMedian,AwayTeamGolsContraMax,AwayTeam_Position,AwayTeam_Home_Wins_season,AwayTeam_Away_Wins_season,AwayTeam_Wins_season,AwayTeam_Home_Lose_season,AwayTeam_Away_Lose_season,AwayTeam_Lose_season,AwayTeam_Home_Draw_season,AwayTeam_Away_Draw_season,AwayTeam_Draw_season,AwayTeam_Points,AwayTeam_Home_goals_avg,AwayTeam_Home_goals_med,AwayTeam_Home_goals_max,AwayTeam_Home_goals_mode,AwayTeam_Away_goals_avg,AwayTeam_Away_goals_med,AwayTeam_Away_goals_max,AwayTeam_Away_goals_mode,AwayTeam_AgeMean,AwayTeam_AgeMedian,AwayTeam_DefensaMax,AwayTeam_DefensaMean,AwayTeam_DefensaMedian,AwayTeam_DelanteroMax,AwayTeam_DelanteroMean,AwayTeam_DelanteroMedian,AwayTeam_MedioMax,AwayTeam_MedioMean,AwayTeam_MedioMedian,AwayTeam_OverallMax,AwayTeam_OverallMean,AwayTeam_OverallMedian,AwayTeam_PorteroMax,AwayTeam_PorteroMean,AwayTeam_PorteroMedian,AwayTeam_TopOverallMean,AwayTeam_TopOverallMedian,AwayTeam_TopOverallMin
0,0.000000,2.0,2.0,2,6.0,6.0,6,10,6,5,11,7,8,15,6,6,12,45,0.894737,1.0,2,1,0.894737,1.0,3,1,26.142857,25.0,78.0,73.555556,73.0,81.0,71.400000,75.0,79.0,74.818182,76.0,81.0,73.678571,75.5,81.0,73.666667,70.0,78.181818,78.0,60.0,1.000000,2.0,2.0,2,1.0,1.0,1,15,4,4,8,7,5,12,8,10,18,42,1.315789,1.0,3,2,1.052632,1.0,3,1,25.666667,25.0,79.0,74.625000,75.0,79.0,68.500000,67.0,79.0,72.062500,74.0,79.0,72.151515,75.0,75.0,73.333333,75.0,77.272727,77.0,61.0
1,0.000000,4.0,4.0,4,6.0,6.0,6,14,9,3,12,5,14,19,5,2,7,43,1.157895,1.0,4,1,0.947368,1.0,4,0,26.142857,26.0,77.0,73.666667,76.0,80.0,73.000000,75.0,79.0,75.300000,77.0,82.0,74.250000,76.5,82.0,75.000000,79.0,78.454545,78.0,64.0,0.333333,1.0,1.0,1,1.0,1.0,1,9,8,4,12,5,9,14,6,6,12,48,1.368421,1.0,4,1,0.631579,0.0,3,0,25.000000,24.0,78.0,73.181818,75.0,77.0,68.875000,71.0,81.0,75.909091,76.0,81.0,73.090909,75.0,81.0,73.666667,78.0,78.272727,78.0,58.0
2,0.000000,1.0,1.0,1,2.0,2.0,2,13,8,3,11,6,11,17,5,5,10,43,1.368421,1.0,5,1,1.210526,1.0,4,0,25.096774,25.0,79.0,72.000000,75.0,78.0,70.500000,72.5,79.0,73.900000,77.0,79.0,71.870968,75.0,75.0,68.500000,69.5,77.545455,78.0,60.0,0.000000,2.0,2.0,2,4.0,4.0,4,12,6,5,11,6,10,16,7,4,11,44,1.315789,1.0,4,0,1.105263,1.0,5,0,24.333333,24.0,83.0,74.166667,76.5,83.0,76.375000,76.5,81.0,71.400000,72.5,83.0,73.878788,76.0,82.0,74.333333,75.0,80.181818,79.0,61.0
3,1.000000,1.0,1.0,1,0.0,0.0,0,19,6,3,9,6,14,20,7,2,9,36,1.210526,1.0,4,1,0.736842,0.0,2,0,25.454545,25.0,76.0,70.909091,74.0,73.0,69.500000,71.0,77.0,73.500000,74.0,77.0,71.939394,73.0,76.0,71.750000,73.0,76.181818,76.0,61.0,0.333333,1.0,1.0,1,1.0,1.0,1,3,15,13,28,1,5,6,3,1,4,88,1.736842,1.0,5,1,1.578947,1.0,3,1,25.419355,24.0,88.0,77.666667,82.0,88.0,75.666667,80.0,84.0,76.416667,80.5,88.0,76.548387,81.0,87.0,75.750000,75.5,84.636364,84.0,61.0
4,0.333333,1.0,1.0,1,1.0,1.0,1,20,7,1,8,7,15,22,5,3,8,32,1.210526,1.0,3,1,0.736842,1.0,3,0,26.030303,27.0,75.0,71.727273,72.0,76

In [0]:
X.describe()

,HomeTeamPercPoints,HomeTeamGolsFavorMean,HomeTeamGolsFavorMedian,HomeTeamGolsFavorMax,HomeTeamGolsContraMean,HomeTeamGolsContraMedian,HomeTeamGolsContraMax,HomeTeam_Position,HomeTeam_Home_Wins_season,HomeTeam_Away_Wins_season,HomeTeam_Wins_season,HomeTeam_Home_Lose_season,HomeTeam_Away_Lose_season,HomeTeam_Lose_season,HomeTeam_Home_Draw_season,HomeTeam_Away_Draw_season,HomeTeam_Draw_season,HomeTeam_Points,HomeTeam_Home_goals_avg,HomeTeam_Home_goals_med,HomeTeam_Home_goals_max,HomeTeam_Home_goals_mode,HomeTeam_Away_goals_avg,HomeTeam_Away_goals_med,HomeTeam_Away_goals_max,HomeTeam_Away_goals_mode,HomeTeam_AgeMean,HomeTeam_AgeMedian,HomeTeam_DefensaMax,HomeTeam_DefensaMean,HomeTeam_DefensaMedian,HomeTeam_DelanteroMax,HomeTeam_DelanteroMean,HomeTeam_DelanteroMedian,HomeTeam_MedioMax,HomeTeam_MedioMean,HomeTeam_MedioMedian,HomeTeam_OverallMax,HomeTeam_OverallMean,HomeTeam_OverallMedian,HomeTeam_PorteroMax,HomeTeam_PorteroMean,HomeTeam_PorteroMedian,HomeTeam_TopOverallMean,HomeTeam_TopOverallMedian,HomeTeam_TopOverallMin,AwayTeamPercPoints,AwayTeamGolsFavorMean,AwayTeamGolsFavorMedian,AwayTeamGolsFavorMax,AwayTeamGolsContraMean,AwayTeamGolsContraMedian,AwayTeamGolsContraMax,AwayTeam_Position,AwayTeam_Home_Wins_season,AwayTeam_Away_Wins_season,AwayTeam_Wins_season,AwayTeam_Home_Lose_season,AwayTeam_Away_Lose_season,AwayTeam_Lose_season,AwayTeam_Home_Draw_season,AwayTeam_Away_Draw_season,AwayTeam_Draw_season,AwayTeam_Points,AwayTeam_Home_goals_avg,AwayTeam_Home_goals_med,AwayTeam_Home_goals_max,AwayTeam_Home_goals_mode,AwayTeam_Away_goals_avg,AwayTeam_Away_goals_med,AwayTeam_Away_goals_max,AwayTeam_Away_goals_mode,AwayTeam_AgeMean,AwayTeam_AgeMedian,AwayTeam_DefensaMax,AwayTeam_DefensaMean,AwayTeam_DefensaMedian,AwayTeam_DelanteroMax,AwayTeam_DelanteroMean,AwayTeam_DelanteroMedian,AwayTeam_MedioMax,AwayTeam_MedioMean,AwayTeam_MedioMedian,AwayTeam_OverallMax,AwayTeam_OverallMean,AwayTeam_OverallMedian,AwayTeam_PorteroMax,AwayTeam_PorteroMean,AwayTeam_PorteroMedian,AwayTeam_TopOverallMean,AwayTeam_TopOverallMedian,AwayTeam_TopOverallMin
count,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000
mean,0.450345,1.337580,1.173478,3.608841,1.356764,1.170559,3.762302,10.403670,9.054212,5.505421,14.559633,5.386155,9.046706,14.432861,4.559633,4.447873,9.007506,52.686405,1.589044,1.451209,4.231026,1.177648,1.177692,1.049208,3.617181,0.688907,25.184992,24.843203,80.154295,73.990881,75.724771,81.135113,74.047992,75.091326,81.087573,74.504779,75.872811,82.574646,73.985456,75.939116,79.761468,72.892369,73.620100,79.376526,79.040867,61.557965,0.458098,1.359763,1.190575,3.638866,1.342859,1.167640,3.739783,10.355296,9.055046,5.529608,14.584654,5.372811,9.007506,14.380317,4.572143,4.462886,9.035029,52.788991,1.593960,1.456214,4.237698,1.183486,1.180633,1.050042,3.628023,0.688073,25.183959,24.838198,80.201835,74.018919,75.761468,81.185154,74.108389,75.164304,81.129274,74.530703,75.879900,82.608007,74.017712,75.967890,79.7

In [0]:
y = data.iloc[:,:4]

In [0]:
y.head()

,B365H,B365D,B365A,FTR
0,0.431766,0.296839,0.271396,D
1,0.413495,0.306787,0.279718,D
2,0.353911,0.298613,0.347476,H
3,0.112150,0.264798,0.623053,D
4,0.308642,0.308642,0.382716,A


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [0]:
y_train = y_train['FTR']

In [0]:
xg_reg = xgb.XGBClassifier()

In [0]:
print(xg_reg)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)


In [0]:

xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)
probs = xg_reg.predict_proba(X_test)


In [0]:
acc = sum(preds == y_test['FTR'].values)/len(preds)
print("Accuracy: %f" % (acc))

Accuracy: 0.545833


In [0]:
res = y_test.copy().reset_index(drop=True)

In [0]:
aux = pd.DataFrame(probs)
res['P_B365H'] = aux[2]
res['P_B365D'] = aux[1]
res['P_B365A'] = aux[0]

In [0]:
res.describe()

,B365H,B365D,B365A,P_B365H,P_B365D,P_B365A
count,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000
mean,0.455792,0.245057,0.299152,0.482911,0.243522,0.273567
std,0.202867,0.058242,0.186481,0.225901,0.132268,0.193988
min,0.073229,0.063585,0.023375,0.024205,0.027739,0.010455
25%,0.328522,0.219644,0.173013,0.320933,0.138775,0.135273
50%,0.454444,0.265293,0.263616,0.487651,0.237533,0.224420
75%,0.589044,0.288217,0.379637,0.656491,0.314294,0.360563
max,0.912734,0.328953,0.798526,0.946203,0.708972,0.911382


In [0]:
rmse = np.sqrt(mean_squared_error(res[['B365H','B365D','B365A']], res[['P_B365H','P_B365D','P_B365A']]))
print("RMSE: %f" % (rmse))

RMSE: 0.115911


In [3]:
math.sqrt(0.17*0.17*+0.03*0.03+0.2*0.2)

0.20006501443280883

In [4]:
math.sqrt(0.014*0.014*+0.011*0.011+0.027*0.027)

0.027000439181613323

In [0]:
math.sqrt(0.006*0.006*+0.003*0.003+0.006*0.027)